In [ ]:
# default_exp leaderboard

# Leaderboard

> Current leaderboard using validation set. The final leaderboard will be generated at the end of the contest using test dataset and will probably be different due to overfitting.

In [ ]:
#exporti

import pandas as pd
import boto3
from pathlib import Path

In [ ]:
# export

def get_leaderboard():
    df = pd.DataFrame(dict(alias=["davor", "matija", "leptirić"], score=[0.483, 0.453, 0.526]))

    return df.sort_values(by="score", ascending=False).reset_index(drop=True)

In [ ]:
#hide_input

get_leaderboard()

,alias,score
0,leptirić,0.526
1,davor,0.483
2,matija,0.453


In [ ]:
df = pd.DataFrame(dict(alias=["davor", "matija", "leptirić"], score=[0.483, 0.453, 0.526], ))
#df.to_csv ("tu_spremi", index = True, header=True)


In [ ]:
import numpy as np

d = dict( A = np.array([1,2]), B = np.array([1,2,3,4]))
    
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items()]))

,A,B
0,1.0,1
1,2.0,2
2,NaN,3
3,NaN,4


In [ ]:
#load or create default csv

d = dict(
    file_name=[
        "uploaded-2020-12-22T15:05:27.814695-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-22T15:05:18.314765.zip",
        "uploaded-2020-12-22T15:35:15.513570-submission-iou=0.46613-dolphin123-name.surname@gmail.com-2020-12-22T15:35:04.875962.zip",
        "uploaded-2020-12-23T07:13:27.094619-submission-iou={iou:.5f}-{alias}-{email}-{ts}.zip",
        "uploaded-2020-12-23T07:19:47.304333-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:19:46.141351.zip",
        "uploaded-2020-12-23T07:21:57.555841-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:21:56.693355.zip",
        "uploaded-2020-12-23T07:30:07.549985-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:30:06.348322.zip",
        "uploaded-2020-12-23T07:30:41.228682-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:30:40.504739.zip",
        "uploaded-2020-12-23T07:31:59.412242-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:31:58.666165.zip",
        "uploaded-2020-12-23T09:20:49.505834-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T09:20:48.705296.zip",
        "uploaded-2020-12-23T09:22:08.692268-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T09:22:07.805722.zip",
        "uploaded-2020-12-23T09:26:56.245937-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T09:26:55.434584.zip",
        "uploaded-2020-12-23T09:28:17.949335-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T09:28:17.113860.zip",
        "uploaded-2020-12-23T10:02:48.457062-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T10:02:47.169365.zip",
        "uploaded-2020-12-23T10:04:05.929944-submission-iou=0.00000-mali medo-matija.krivosic@gmail.com-2020-12-23T10:04:05.151063.zip",
        "uploaded-2020-12-23T10:30:24.725200-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T10:30:23.416720.zip",
        "uploaded-2020-12-23T10:33:28.700064-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T10:33:27.929936.zip",
        "uploaded-2020-12-23T11:11:35.661267-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T11:11:34.391507.zip",
        "uploaded-2020-12-23T11:46:57.087575-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T11:46:55.682793.zip",
        "uploaded-2020-12-23T11:48:16.983091-submission-iou=0.45325-mali medo-matija.krivosic@gmail.com-2020-12-23T11:48:16.144154.zip",
        "uploaded-2020-12-23T12:28:50.479516-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T12:28:49.383147.zip",
        "uploaded-2020-12-23T12:52:45.307554-submission-iou=0.00000-dolphin123-name.surname@gmail.com-2020-12-23T12:52:43.545142.zip",
        "uploaded-2020-12-24T12:48:48.699591-submission-iou=0.45325-mali medo-matija.krivosic@gmail.com-2020-12-24T12:48:44.730529.zip",
        "uploaded-2020-12-24T12:48:59.145237-submission-iou=0.45325-mali medo-matija.krivosic@gmail.com-2020-12-24T12:48:55.574652.zip",
        "uploaded-2021-01-04T09:17:57.326254-submission-iou=0.45722-dolphin123-name.surname@gmail.com-2021-01-04T09:17:55.902567.zip",
        "uploaded-2021-01-05T10:38:27.937514-submission-iou=0.44608-dolphin123-name.surname@gmail.com-2021-01-05T10:38:26.133472.zip",
        "uploaded-2021-01-05T10:40:28.088536-submission-iou=0.45344-dolphin123-name.surname@gmail.com-2021-01-05T10:40:26.687938.zip",
        "uploaded-2021-01-05T12:40:06.758744-submission-iou=0.45871-dolphin123-name.surname@gmail.com-2021-01-05T12:40:05.356274.zip",
    ],
    slova=np.array(["a", "b"]),
    brojevi=np.array([0, 1]),
)

file_names_dataframe = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in d.items()]))
file_names_dataframe

,file_name,slova,brojevi
0,uploaded-2020-12-22T15:05:27.814695-submission...,a,0.0
1,uploaded-2020-12-22T15:35:15.513570-submission...,b,1.0
2,uploaded-2020-12-23T07:13:27.094619-submission...,NaN,NaN
3,uploaded-2020-12-23T07:19:47.304333-submission...,NaN,NaN
4,uploaded-2020-12-23T07:21:57.555841-submission...,NaN,NaN
5,uploaded-2020-12-23T07:30:07.549985-submission...,NaN,NaN
6,uploaded-2020-12-23T07:30:41.228682-submission...,NaN,NaN
7,uploaded-2020-12-23T07:31:59.412242-submission...,NaN,NaN
8,uploaded-2020-12-23T09:20:49.505834-submission...,NaN,NaN
9,uploaded-2020-12-23T09:22:08.692268-submission...,NaN,NaN


In [ ]:
file_names_dataframe[["0", "year","month","day_time","4","score","alias","email","8","9","10" ]] = file_names_dataframe.file_name.str.split('-', expand=True)



In [ ]:
"""Load csv file"""

'Load csv file'

In [ ]:
to_show_dataframe = file_names_dataframe[["file_name", "year", "month", "day_time", "score", "alias", "email"]]

In [ ]:
to_show_dataframe[['day','time']] = to_show_dataframe.day_time.str.split("T",expand=True,)
to_show_dataframe

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,file_name,year,month,day_time,score,alias,email,day,time
0,uploaded-2020-12-22T15:05:27.814695-submission...,2020,12,22T15:05:27.814695,iou=0.44641,mali medo,matija.krivosic@gmail.com,22,15:05:27.814695
1,uploaded-2020-12-22T15:35:15.513570-submission...,2020,12,22T15:35:15.513570,iou=0.46613,dolphin123,name.surname@gmail.com,22,15:35:15.513570
2,uploaded-2020-12-23T07:13:27.094619-submission...,2020,12,23T07:13:27.094619,iou={iou:.5f},{alias},{email},23,07:13:27.094619
3,uploaded-2020-12-23T07:19:47.304333-submission...,2020,12,23T07:19:47.304333,iou=0.44641,mali medo,matija.krivosic@gmail.com,23,07:19:47.304333
4,uploaded-2020-12-23T07:21:57.555841-submission...,2020,12,23T07:21:57.555841,iou=0.44641,mali medo,matija.krivosic@gmail.com,23,07:21:57.555841
5,uploaded-2020-12-23T07:30:07.549985-submission...,2020,12,23T07:30:07.549985,iou=0.44641,mali medo,matija.krivosic@gmail.com,23,07:30:07.549985
6,uploaded-2020-12-23T07:30:41.228682-submission...,2020,12,23T07:30:41.228682,iou=0.44641,mali medo,matija.krivosic@gmail.com,23,07:30:41.228682
7,uploaded-2020-12-23T07:31:59.412242-submission...,2020,12,23T07:31:59.412242,iou=0.44641,mali medo,matija.krivosic@gmail.com,23,07:31:59.412242
8,uploaded-2020-12-23T09:20:49.505834-submission...,2020,12,23T09:20:49.505834,iou=0.00000,dolphin123,name.surname@gmail.com,23,09:20:49.505834
9,uploaded-2020-12-23T09:22:08.692268-submission...,2020,12,23T09:22:08.692268,iou=0.44641,mali medo,matija.krivosic@gmail.com,23,09:22:08.692268


In [ ]:
to_show_dataframe = to_show_dataframe[["file_name","year", "month", "day", "time", "score", "alias", "email"]]
to_show_dataframe

,file_name,year,month,day,time,score,alias,email
0,uploaded-2020-12-22T15:05:27.814695-submission...,2020,12,22,15:05:27.814695,iou=0.44641,mali medo,matija.krivosic@gmail.com
1,uploaded-2020-12-22T15:35:15.513570-submission...,2020,12,22,15:35:15.513570,iou=0.46613,dolphin123,name.surname@gmail.com
2,uploaded-2020-12-23T07:13:27.094619-submission...,2020,12,23,07:13:27.094619,iou={iou:.5f},{alias},{email}
3,uploaded-2020-12-23T07:19:47.304333-submission...,2020,12,23,07:19:47.304333,iou=0.44641,mali medo,matija.krivosic@gmail.com
4,uploaded-2020-12-23T07:21:57.555841-submission...,2020,12,23,07:21:57.555841,iou=0.44641,mali medo,matija.krivosic@gmail.com
5,uploaded-2020-12-23T07:30:07.549985-submission...,2020,12,23,07:30:07.549985,iou=0.44641,mali medo,matija.krivosic@gmail.com
6,uploaded-2020-12-23T07:30:41.228682-submission...,2020,12,23,07:30:41.228682,iou=0.44641,mali medo,matija.krivosic@gmail.com
7,uploaded-2020-12-23T07:31:59.412242-submission...,2020,12,23,07:31:59.412242,iou=0.44641,mali medo,matija.krivosic@gmail.com
8,uploaded-2020-12-23T09:20:49.505834-submission...,2020,12,23,09:20:49.505834,iou=0.00000,dolphin123,name.surname@gmail.com
9,uploaded-2020-12-23T09:22:08.692268-submission...,2020,12,23,09:22:08.692268,iou=0.44641,mali medo,matija.krivosic@gmail.com


In [ ]:
to_show_dataframe[['score_evaluated','score']] = to_show_dataframe.score.str.split("=",expand=True,)
to_show_dataframe

,file_name,year,month,day,time,score,alias,email,score_evaluated
0,uploaded-2020-12-22T15:05:27.814695-submission...,2020,12,22,15:05:27.814695,0.44641,mali medo,matija.krivosic@gmail.com,iou
1,uploaded-2020-12-22T15:35:15.513570-submission...,2020,12,22,15:35:15.513570,0.46613,dolphin123,name.surname@gmail.com,iou
2,uploaded-2020-12-23T07:13:27.094619-submission...,2020,12,23,07:13:27.094619,{iou:.5f},{alias},{email},iou
3,uploaded-2020-12-23T07:19:47.304333-submission...,2020,12,23,07:19:47.304333,0.44641,mali medo,matija.krivosic@gmail.com,iou
4,uploaded-2020-12-23T07:21:57.555841-submission...,2020,12,23,07:21:57.555841,0.44641,mali medo,matija.krivosic@gmail.com,iou
5,uploaded-2020-12-23T07:30:07.549985-submission...,2020,12,23,07:30:07.549985,0.44641,mali medo,matija.krivosic@gmail.com,iou
6,uploaded-2020-12-23T07:30:41.228682-submission...,2020,12,23,07:30:41.228682,0.44641,mali medo,matija.krivosic@gmail.com,iou
7,uploaded-2020-12-23T07:31:59.412242-submission...,2020,12,23,07:31:59.412242,0.44641,mali medo,matija.krivosic@gmail.com,iou
8,uploaded-2020-12-23T09:20:49.505834-submission...,2020,12,23,09:20:49.505834,0.00000,dolphin123,name.surname@gmail.com,iou
9,uploaded-2020-12-23T09:22:08.692268-submission...,2020,12,23,09:22:08.692268,0.44641,mali medo,matija.krivosic@gmail.com,iou


In [ ]:
# exporti

def get_file_names(dataf, column="file_name"):
    return list(dataf[column])

In [ ]:
# hide

get_file_names(to_show_dataframe)

['uploaded-2020-12-22T15:05:27.814695-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-22T15:05:18.314765.zip',
 'uploaded-2020-12-22T15:35:15.513570-submission-iou=0.46613-dolphin123-name.surname@gmail.com-2020-12-22T15:35:04.875962.zip',
 'uploaded-2020-12-23T07:13:27.094619-submission-iou={iou:.5f}-{alias}-{email}-{ts}.zip',
 'uploaded-2020-12-23T07:19:47.304333-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:19:46.141351.zip',
 'uploaded-2020-12-23T07:21:57.555841-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:21:56.693355.zip',
 'uploaded-2020-12-23T07:30:07.549985-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:30:06.348322.zip',
 'uploaded-2020-12-23T07:30:41.228682-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:30:40.504739.zip',
 'uploaded-2020-12-23T07:31:59.412242-submission-iou=0.44641-mali medo-matija.krivosic@gmail.com-2020-12-23T07:31:58.666165.zip',
 '

In [ ]:
# hide

download_path = Path("models_for_evaluation")
if not download_path.exists():
    download_path.mkdir()

In [ ]:
# hide

path_1 = Path("xy/file.zip")
path_2 = Path("xy/xy")

print(path_1.match("*.zip"))
print(path_2.match("*.zip"))

True
False


In [ ]:
s3 = boto3.resource("s3")
my_bucket = s3.Bucket("ai-league.cisex.org")

In [ ]:
# exporti


def download_models(s3, my_bucket, dataframe):
    list_of_models = []

    # download file into models_for_evaluation directory
    for s3_object in my_bucket.objects.all():
        path = Path(s3_object.key)
        if path.match("*submission*.zip") and path.name not in get_file_names(
            dataframe
        ):  # and not in csv
            print(path.name)
            print(path)
            list_of_models.append(path.name)
            my_bucket.download_file(
                str(path), "models_for_evaluation/" + str(path.name)
            )

In [ ]:
#skip

download_models(s3, my_bucket, to_show_dataframe)

uploaded-2021-01-05T15:01:23.563795-submission-iou=0.44003-dolphin123-name.surname@gmail.com-2021-01-05T15:01:21.655750.zip
2020-2021/dolphins-instance-segmentation/submissions/uploaded-2021-01-05T15:01:23.563795-submission-iou=0.44003-dolphin123-name.surname@gmail.com-2021-01-05T15:01:21.655750.zip


In [ ]:
#create csv file test
to_show_dataframe.to_csv(r'leaderboard.csv', index = False, header=True)

In [ ]:
#load csv file test
frame = pd.read_csv("leaderboard.csv")

In [ ]:
frame

,file_name,year,month,day,time,score,alias,email,score_evaluated
0,uploaded-2020-12-22T15:05:27.814695-submission...,2020,12,22,15:05:27.814695,0.44641,mali medo,matija.krivosic@gmail.com,iou
1,uploaded-2020-12-22T15:35:15.513570-submission...,2020,12,22,15:35:15.513570,0.46613,dolphin123,name.surname@gmail.com,iou
2,uploaded-2020-12-23T07:13:27.094619-submission...,2020,12,23,07:13:27.094619,{iou:.5f},{alias},{email},iou
3,uploaded-2020-12-23T07:19:47.304333-submission...,2020,12,23,07:19:47.304333,0.44641,mali medo,matija.krivosic@gmail.com,iou
4,uploaded-2020-12-23T07:21:57.555841-submission...,2020,12,23,07:21:57.555841,0.44641,mali medo,matija.krivosic@gmail.com,iou
5,uploaded-2020-12-23T07:30:07.549985-submission...,2020,12,23,07:30:07.549985,0.44641,mali medo,matija.krivosic@gmail.com,iou
6,uploaded-2020-12-23T07:30:41.228682-submission...,2020,12,23,07:30:41.228682,0.44641,mali medo,matija.krivosic@gmail.com,iou
7,uploaded-2020-12-23T07:31:59.412242-submission...,2020,12,23,07:31:59.412242,0.44641,mali medo,matija.krivosic@gmail.com,iou
8,uploaded-2020-12-23T09:20:49.505834-submission...,2020,12,23,09:20:49.505834,0.00000,dolphin123,name.surname@gmail.com,iou
9,uploaded-2020-12-23T09:22:08.692268-submission...,2020,12,23,09:22:08.692268,0.44641,mali medo,matija.krivosic@gmail.com,iou


In [ ]:
modelas_path = Path("models_for_evaluation")
list_of_models_to_eval = [x for x in modelas_path.glob("*.zip")]
print(list_of_models_to_eval)

[PosixPath('models_for_evaluation/uploaded-2021-01-05T15:01:23.563795-submission-iou=0.44003-dolphin123-name.surname@gmail.com-2021-01-05T15:01:21.655750.zip')]


In [ ]:
import zipfile

for zip_file in list_of_models_to_eval:
    with zipfile.ZipFile(zip_file,"r") as zip_ref:
        zip_ref.extractall("models_for_evaluation")


In [ ]:
list_of_dir = [x for x in modelas_path.glob("*") if x.name.startswith("submission")]
print(list_of_dir)
for directory in list_of_dir:
    


[PosixPath('models_for_evaluation/submission-iou=0.44003-dolphin123-name.surname@gmail.com-2021-01-05T15:01:21.655750')]


In [ ]:
#! rm -rf "models_for_evaluation/submission-iou=0.44003-dolphin123-name.surname@gmail.com-2021-01-05T15:01:21.655750"